# Cleaning and Preparing Data

## Problem Statement

#### When is the best time to release a new song/album on Spotify?

As a major label artist, planning and preparing a successful release goes further than finding a great single or recording a critically acclaimed album. A strategic rollout is essential to standing out and cutting through noise in this highly competitive industry. The target of this project is to pinpoint an effective time to release new music.

In order to achieve this, I decided to gather data from the largest music streaming service, Spotify, along with chart data from the industry publication Billboard to analyze the songs that see the most success while on the best selling/most listened to charts.


This notebook walks through the processes of preparing the data collected from Billboard and Spotify and calling additional data from Spotify to prepare for exploratory analysis and projecting release seasons. Sources can be found throughout the notebook.

In [1]:
# Importing packages that will be used to organize data gathered from Spotify and Billboard.
# In order to reproduce results with Spotify data, a request for and Spotify ID and Secret key need to be requested.
# Using a spotify account, log on to https://developer.spotify.com/dashboard/login

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

spotify_id = #'your spotify key here'
spotify_secret = #'your spotify secret key here'
spotify_url = 'https://api.spotify.com/v1/'

client_credentials_manager = SpotifyClientCredentials(client_id=spotify_id, client_secret=spotify_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
# Loading in data collected from Billboard and Spotify
# For that process, refer to '01 Data Gathering'

hot_100 = pd.read_csv('data/hot_100.csv')
spotify_200 = pd.read_csv('data/spotify_stream_totals.csv')
spotify_peak_chart = pd.read_csv('data/spotify_chart_positions.csv')

In [3]:
hot_100.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,Old Town Road,If I Can't Have You,ME!,Sucker,Homicide,Wow.,Sunflower (Spider-Man: Into The Spider-Verse),Without Me,Bad Guy,7 Rings,...,Middle Child,Shot Clock,Ocean Eyes,Wish You Were Gay,I've Been Waiting,All To Myself,Secreto,Paradise,Die Young,Faucet Failure
1,Old Town Road,ME!,Wow.,Sucker,Sunflower (Spider-Man: Into The Spider-Verse),7 Rings,Without Me,Dancing With A Stranger,Bad Guy,Talk,...,Love Someone,Power Is Power,Faucet Failure,Knockin' Boots,Numb Numb Juice,Love Me Anyway,24/7,Stop Snitching,SOS,Kill This Love
2,Old Town Road,Wow.,Sunflower (Spider-Man: Into The Spider-Verse),7 Rings,Sucker,Without Me,Dancing With A Stranger,Talk,Bad Guy,Middle Child,...,Saturday Nights,Pure Cocaine,Love Someone,This Is It,My Strange Addiction,Shotta Flow,Let Me Down Slowly,Xanny,One That Got Away,ME!
3,Old Town Road,Wow.,Sunflower (Spider-Man: Into The Spider-Verse),7 Rings,Without Me,Sucker,Dancing With A Stranger,Boy With Luv,Bad Guy,Please Me,...,Ocean Eyes,Last Time That I Checc'd,Talk You Out Of It,I'm So Tired...,Make It Right,"Hey Look Ma, I Made It",This Is It,There Was This Girl,Let Me Down Slowly,On My Way To You
4,Old Town Road,Sunflower (Spider-Man: Into The Spider-Verse),Wow.,7 Rings,Without Me,Sucker,Please Me,Better,Middle Child,Happier,...,Faucet Failure,All The Good Girls Go To Hell,Right Back,Talk You Out Of It,Inmortal,Ocean Eyes,Dedication,Let Me Down Slowly,Big Ole Freak,Victory Lap


In [4]:
# Need to remove repeat values from hot_100 chart data. Also need to get the peak position for reference
# Loop initially adapted from 
# https://stackoverflow.com/questions/8200342/removing-duplicate-strings-from-a-list-in-python

seen = set()
hot_100_unique = []          # Holds name of song 
hot_100_peak = []            # Holds the peak position of track in corresponding hot_100_unique list
for n in range(1, 101):
    for item in hot_100.loc[:][f'{n}']:
        if item not in seen:
            seen.add(item)
            hot_100_unique.append(item)
            hot_100_peak.append(n)

In order to get all the data needed for modeling, the chart data from spotify had to be organized in two different ways from its original format.

First, the data has to be organized by the chart position by week. Once ordered, the peak position can be sourced from this data. Next, the number of streams for each time the song appears on the top 200 chart need to be added up. 

The following code moves through this process of creating the complete dataframe.

In [5]:
# Organizing spotify_peak_chart to track highest position for each track

spotify_peak_chart = spotify_peak_chart.sort_values(by=['Position'])

spotify_peak_chart.reset_index(inplace=True)

spotify_peak_chart.drop(['index'], axis=1, inplace=True)

In [8]:
# Need to remove repeat values from spotify_peak_chart data. Also need to get the peak position for reference
# Loop initially adapted from 
# https://stackoverflow.com/questions/8200342/removing-duplicate-strings-from-a-list-in-python

seen = set()
spotify_200_unique = []          # Holds name of song 
spotify_200_peak = []            # Holds the peak position of track in corresponding spotify_200_unique
for n in range(0, 24400):
    item = spotify_peak_chart.loc[n]
    if item['Track Name'] not in seen:
        seen.add(item['Track Name'])
        spotify_200_unique.append(item['Track Name'])
        spotify_200_peak.append(item['Position'])

In [9]:
spotify_peak_chart_unique = pd.DataFrame(data= spotify_200_unique, 
                                         columns = ['Track Name'])
spotify_peak_chart_unique['Spotify Peak Pos'] = spotify_200_peak

spotify_peak_chart_unique = spotify_peak_chart_unique.sort_values(['Track Name']).reset_index()
spotify_peak_chart_unique.drop(['index'], axis=1, inplace=True)
spotify_peak_chart_unique.head()

,Track Name,Spotify Peak Pos
0,$$$ - with Matt Ox,33
1,'Till I Collapse,137
2,(Fuck A) Silver Lining,128
3,...Ready For It?,6
4,1 SIDED LOVE,194


In [10]:
spotify_200 = spotify_200.sort_values(['Track Name'])

In [11]:
spotify_200.reset_index(inplace=True)

In [12]:
df = pd.DataFrame(spotify_200['Streams'].groupby(spotify_200['Track Name']).sum()).reset_index()

In [13]:
df['Spotify Peak Pos'] = spotify_peak_chart_unique['Spotify Peak Pos']

In [14]:
df.head()

,Track Name,Streams,Spotify Peak Pos
0,$$$ - with Matt Ox,12265622,33
1,'Till I Collapse,42085250,137
2,(Fuck A) Silver Lining,3281981,128
3,...Ready For It?,56087780,6
4,1 SIDED LOVE,1525582,194


Unfortunately, Billboard does not disclose sales data compiled from Nielsen Soundscan reports that are used to create these charts, as I originally thought. This meant I would only be able to use Spotify streaming data for the project. Future iterations of this model and project would include access to Nielsen Soundscan's proprietary data to create a more accurate model.

Below the process for combining the Spotify track data and corresponding billboard chart position is laid out.

In [15]:
working = pd.DataFrame(hot_100_unique, columns=['Track Name'])

In [16]:
working['Peak Pos'] = hot_100_peak

In [17]:
working.head()

,Track Name,Peak Pos
0,Old Town Road,1
1,7 Rings,1
2,Sucker,1
3,Shallow,1
4,Without Me,1


In [19]:
len(working)

1302

In [20]:
# Matching billboard track name with spotify track name
# Adapted from: https://stackoverflow.com/questions/3437059/does-python-have-a-string-contains-substring-method?rq=1
bb_peak_pos = []
artists_in_both_dfs = []
stream_totals = []
sp_peak_pos = []
for i in range(0, 2356):
    s = df['Track Name'][i]
    for n in range(0, 1302):
        if s.find(working['Track Name'][n]) != 0:
            pass
        else:
            bb_peak_pos.append(working['Peak Pos'][n])
            artists_in_both_dfs.append(df['Track Name'][i])
            stream_totals.append(df['Streams'][i])
            sp_peak_pos.append(df['Spotify Peak Pos'][i])

In [21]:
# Creating the compiled dataframe

final = pd.DataFrame(artists_in_both_dfs, columns=['track_name'])

The next cells use the track names from above to search the Spotify API for information on the track's performer and genre.

Note:
Ensure spotify key and secret key are entered above before running the next cells

In [22]:
len(final)

1288

In [23]:
# Song info will be a list that contains the dictionaries with info for each track.

song_info = []
for i in range(0, 1288):
    song = sp.search(q=final['track_name'][i], type='track')['tracks']['items']
    song_info.append(song)
    

In [26]:
# Release date and Artist name pulled from dictionaries

release_date = []
artist_name = []
for i in range(0, 1288):
    if len(song_info[i]) == 0:
        release_date.append('NA')
        artist_name.append('NA')
    else:
        date = song_info[i][0]['album']['release_date']
        artist = song_info[i][0]['album']['artists'][0]['name']
        release_date.append(date)
        artist_name.append(artist)
        

In [27]:
# Using the artist's name, information was pulled from the API including genre and artist popularity.

artist_info = []
for i in range(0, 1288):
    artists = sp.search(q=artist_name[i], type='artist')['artists']['items'][0]
    artist_info.append(artists)

retrying ...2secs
retrying ...1secs
retrying ...3secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...2secs
retrying ...1secs


In [28]:
# Pulling the items needed from the artist info dictionaries. If information from the API was not retrieved,
# an NA was listed and will be dropped later.

artist_followers = []
artist_genre = []
artist_popularity = []
for i in range(0, 1288):
    if len(artist_info[i]) == 0:
        artist_followers.append('NA')
        artist_genre.append('NA')
        artist_popularity.append('NA')
    else:
        follows = artist_info[i]['followers']['total']
        genre = artist_info[i]['genres']
        popular = artist_info[i]['popularity']
        artist_followers.append(follows)
        artist_genre.append(genre)
        artist_popularity.append(popular)

In [29]:
# Adding all the information to the dataframe that will be used for modeling.

final['artist_name'] = artist_name
final['released_on'] = release_date
final['streams'] = stream_totals
final['bb_hot_100_peak'] = bb_peak_pos
final['sp_top_200_peak'] = sp_peak_pos
final['genres'] = artist_genre
final['artist_popularity'] = artist_popularity
final['artist_followers'] = artist_followers

In [30]:
# Genres from Spotify API were pulled as a list inside each cell. Removing the list and joining into one string

genres_joined = []
for i in range(0, 1288):
    genres_joined.append(' '.join(final['genres'][i]))

In [31]:
final['genres'] = genres_joined

In [32]:
# Some miscelaaneous Various Artists were added on the Spotify Top Charts - removing those and the NA values
# https://stackoverflow.com/questions/28679930/how-to-drop-rows-from-pandas-data-frame-that-contains-a-particular-string-in-a-p

final = final[~final['artist_name'].str.contains('Various Artists')]

final = final[~final['artist_name'].str.contains('NA')]

final.head()

,track_name,artist_name,released_on,streams,bb_hot_100_peak,sp_top_200_peak,genres,artist_popularity,artist_followers
0,...Ready For It?,Taylor Swift,2017-11-10,56087780,4,6,dance pop pop post-teen pop,92,19739479
1,1-800-273-8255,Logic,2017-05-05,316028795,3,1,conscious hip hop dmv rap hip hop pop rap rap,90,3630301
2,1.5,21 Savage,2018-12-21,8834772,86,52,atl hip hop rap trap music,90,6057541
3,10 Freaky Girls (with 21 Savage),Metro Boomin,2018-11-06,68506670,42,11,hip hop pop pop rap rap southern hip hop trap ...,85,1259993
4,1400 / 999 Freestyle,Trippie Redd,2018-11-09,45818520,55,23,rap trap music,86,2947101


In [33]:
final.reset_index(drop=True, inplace=True)

In [34]:
final.head()

,track_name,artist_name,released_on,streams,bb_hot_100_peak,sp_top_200_peak,genres,artist_popularity,artist_followers
0,...Ready For It?,Taylor Swift,2017-11-10,56087780,4,6,dance pop pop post-teen pop,92,19739479
1,1-800-273-8255,Logic,2017-05-05,316028795,3,1,conscious hip hop dmv rap hip hop pop rap rap,90,3630301
2,1.5,21 Savage,2018-12-21,8834772,86,52,atl hip hop rap trap music,90,6057541
3,10 Freaky Girls (with 21 Savage),Metro Boomin,2018-11-06,68506670,42,11,hip hop pop pop rap rap southern hip hop trap ...,85,1259993
4,1400 / 999 Freestyle,Trippie Redd,2018-11-09,45818520,55,23,rap trap music,86,2947101


In [35]:
# Changing the released on to Date/Time format.

final['released_on'] = pd.to_datetime(final['released_on'])

In [36]:
# Using the Date/Time information, pulling out the month of release

month_released = []
for i in range(0, len(final)):
    month_released.append(final['released_on'][i].month)
    
final['month_released'] = month_released

In [37]:
# Saving the final csv for use in 03 EDA and Modeling Notebook

final.to_csv('data/song_data.csv', index=False)